In [23]:
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
from scipy.sparse import coo_matrix
from Image import Image
import os

In [46]:
np.polynomial.hermite.hermgauss(15)


(array([-4.49999071, -3.66995037, -2.96716693, -2.32573249, -1.71999258,
        -1.13611559, -0.56506958,  0.        ,  0.56506958,  1.13611559,
         1.71999258,  2.32573249,  2.96716693,  3.66995037,  4.49999071]),
 array([1.52247580e-09, 1.05911555e-06, 1.00004441e-04, 2.77806884e-03,
        3.07800339e-02, 1.58488916e-01, 4.12028687e-01, 5.64100309e-01,
        4.12028687e-01, 1.58488916e-01, 3.07800339e-02, 2.77806884e-03,
        1.00004441e-04, 1.05911555e-06, 1.52247580e-09]))